In [8]:
!python --version

Python 3.10.0


In [9]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col,lit,call_udf,countDistinct,sproc,udf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from configs_conn.config import snowflake_connect as snowflake_conn
import sys
import json                
import platform
import os,requests
from pathlib import Path
import glob
import pandas as pd
from src.DataValidationContext import *

from snowflake.snowpark import version
print(version.VERSION)

session = Session.builder.configs(snowflake_conn).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(1, 19, 0)
[Row(CURRENT_WAREHOUSE()='COMPUTE_WH', CURRENT_DATABASE()='GX_TRY_DB', CURRENT_SCHEMA()='PUBLIC')]


In [10]:
session.sql("select current_account()").collect()

[Row(CURRENT_ACCOUNT()='AH93068')]

In [11]:
# import os
# import requests
# from pathlib import Path
# import tarfile

# PROJECT_HOME_DIR = '.'
# LOCAL_TEMP_DIR = os.path.join(PROJECT_HOME_DIR, 'temp') 
# LOCAL_LIB_DIR = os.path.join(LOCAL_TEMP_DIR, 'libs')
# LOCAL_TARFile_DIR = os.path.join(LOCAL_TEMP_DIR, 'tarfiles')
# LIB_URLS = [
#     #'https://files.pythonhosted.org/packages/8e/9d/cecb12289f7967b15facf550a0bbb9c1e910968c3a61b91fd8cdb80aeb3c/great_expectations-0.15.14.tar.gz'
#     'https://files.pythonhosted.org/packages/be/be/2be7040e5ff87b33f253e98bd1b18421c0b1027cf46fed31720f8a3a03c9/great_expectations-0.17.11.tar.gz'
# ]

# for lib_url in LIB_URLS:
#     # get the file name, from the url
#     splits = lib_url.split('/')
#     tot_splits = len(splits)
#     target_file = splits[-1]
    
#     local_lib_fl = os.path.join(LOCAL_TARFile_DIR, target_file)
#     print(local_lib_fl)

#     # Create a local directory for TAR and extracting tar..
#     Path(LOCAL_TARFile_DIR).mkdir(parents=True, exist_ok=True)
#     print(f'Created local dir: {LOCAL_TARFile_DIR}')

#     Path(LOCAL_LIB_DIR).mkdir(parents=True, exist_ok=True)
#     print(f'Created local dir: {LOCAL_LIB_DIR}')

#     print(f'Downloading library from PyPI to {LOCAL_TARFile_DIR} ...')
#     with open(local_lib_fl, "wb") as f:
#         r = requests.get(lib_url)
#         f.write(r.content)

# # Extract GE tar file
# try:
#     with tarfile.open(local_lib_fl) as file:
#         print(f'Started extracting GE tar file to {LOCAL_LIB_DIR}/ge ...')
#         file.extractall(os.path.join(LOCAL_LIB_DIR, 'ge'))
#         print(f'Done extracting GE tar file to {LOCAL_LIB_DIR}/ge ...')
# except Exception as e:
#     print(f'Error extracting {local_lib_fl}: {e}')


In [12]:
import glob
ge_import_path=''
for result in glob.iglob('./temp/libs/ge/great_expectations-0.15*'):
    ge_import_path=result+'/great_expectations'
print(ge_import_path)

./temp/libs/ge\great_expectations-0.15.14/great_expectations


In [13]:
from configs_conn.config import snowflake_connect as snowflake_conn
from src.DataValidationContext import GEDataValidationContext
from src.BatchRequest import getBatchRequest 
from src.Expectations import  createExpectationSuite, createExpectations
from src.RunLoadExpectations import runExpectaionValidation,loadValidationToDB


import json
import os
from custom_Expectations.cust_try import ExpectColumnValuesToBeWithinFreshnessExpectation


session.sql("create or replace stage tired").collect()

session.clear_packages()
session.add_packages('pyparsing', 'pandas', 'pydantic==1.10.12', 'makefun==1.15.1', 'ipywidgets==8.1.2', 'pycryptodomex', 'boto3', 'tzlocal', 'tqdm', 'requests', 'ruamel.yaml', 'ipython', 'jsonpatch', 'mistune==2.0.4', 'jinja2', 'jsonschema==4.19.2', 'scipy', 'altair', 'Click', 'colorama', 'cryptography', 'snowflake-snowpark-python', 'sqlalchemy', 'chardet', 'asn1crypto', 'marshmallow')

session.clear_imports()
session.add_import(ge_import_path)
session.add_import('src')
session.add_import('configs_conn')
session.add_import('custom_Expectations')

@sproc(session=session,name="usp_generateGEValidationResults", replace=True, return_type=StringType(),input_types=[StringType(),StringType(),StringType(),StringType()], is_permanent=True, stage_location='@tired/ge_AllLibs')
def generateGEValidationResults(session: Session,datasourcename:str,expecationsuitename:str,checkpointname:str,sftablename) -> str:
    try:
        # Import and register the custom expectation
        from custom_Expectations.cust_try import ExpectColumnValuesToBeWithinFreshnessExpectation
        from great_expectations.expectations.registry import register_expectation
        register_expectation(ExpectColumnValuesToBeWithinFreshnessExpectation)
        
    except Exception as e:
        return f"Error registering custom expectation: {e}"

    from pathlib import Path
    import os ,sys ,json ,tarfile

 
    #Creating GE context inside code
    ge=GEDataValidationContext(datasourcename)
    context=ge.getContext()
    
    # Creating the Pandas DataFrame from Snowpark DF
    pd_df=session.sql("select * from Ge").to_pandas()
    
    # Getting the batch request used while creating and running validation on expectations
    local_batch_request=getBatchRequest(context,datasourcename,pd_df)
    
    #Creating GE expectation Suite
    createExpectationSuite(context,expecationsuitename)
    
    #Creating GE expecations
    createExpectations(context,expecationsuitename,local_batch_request,pd_df)
    
    #Running GE validation 
    res=runExpectaionValidation(context,checkpointname,local_batch_request,expecationsuitename,datasourcename)
    
    #Loading validation result to Snowflake table. Using append option while writing the data to the table
    loadValidationToDB(session,res,sftablename)
      
    return 'SUCCESS'

In [14]:
session.sql("call usp_generateGEValidationResults('PandasDataSource','StudExpecatation','StudCheckpoint','StudGEValidation')").collect()

[Row(USP_GENERATEGEVALIDATIONRESULTS='SUCCESS')]